# Flattening JSON data using Pandas

In [1]:
import os
import json
import pandas as pd
from IPython import display

In [2]:
JSON_FILE = "vertx_example.json"

In [3]:
with open(JSON_FILE, 'r') as jf:
    json_data = json.load(jf)
display.JSON(json_data)

<IPython.core.display.JSON object>

## Straightforward Approach

In [17]:
def load_vertx_data(json_file):
    with open(json_file, 'r') as jf:
        json_data = json.load(jf)
    
    flattened_data = []
    for mte in json_data:
        que_uid = mte.get('source_uid', pd.NA)
        media_type = mte.get('media_type', pd.NA)
        source_path = mte.get('source_path', pd.NA)
        status = mte.get('status', pd.NA)
        for m in mte['matches']:
            artist = m.get('artist', pd.NA)
            album = m.get('album', pd.NA)
            title = m.get('title', pd.NA)
            type_ = m.get('type', pd.NA)
            ref_uid = m.get('uid', pd.NA)
            imdb_id = m.get('imdb_id', pd.NA)
            year = m.get('year', pd.NA)
            for s in m['segments']:
                duration = s['duration']
                que_offset = s['que_offset']
                ref_offset = s['ref_offset']
                entry = {
                    'que_uid': que_uid,
                    'media_type': media_type,
                    'source_path': source_path,
                    'status': status,
                    'artist': artist,
                    'album': album,
                    'title': title,
                    'type': type_,
                    'ref_uid': ref_uid,
                    'imdb_id': imdb_id,
                    'year': year,
                    'que_offset': que_offset,
                    'ref_offset': ref_offset,
                    'duration': duration,
                }
                flattened_data.append(entry)
    
    return pd.DataFrame(flattened_data)

In [18]:
df = load_vertx_data(JSON_FILE)
df.head(5)

,que_uid,media_type,source_path,status,artist,album,title,type,ref_uid,imdb_id,year,que_offset,ref_offset,duration
0,1758730481226206085,audio,sample.mp4,succeeded,Black Eyed Peas; Papa Roach,Elephunk,Anxiety,music,6475547275973858650,<NA>,<NA>,0.0,18.000,24.9375
1,1758730481226206085,audio,sample.mp4,succeeded,<NA>,<NA>,You Got Served,movie,1566530810344932800,365957,2004,0.0,3500.625,24.9375
2,1758730481226206085,video,sample.mp4,succeeded,<NA>,<NA>,Troy,movie,1342729426672482861,332452,2004,0.0,8608.500,21.0000


## Flattening JSON using Pandas

In [19]:
from pandas import json_normalize

In [20]:
def load_vertx_data_using_json_normalize(json_file):
    with open(json_file, 'r') as jf:
        json_data = json.load(jf)
    df = json_normalize(
        json_data,
        record_path=['matches', 'segments'],
        meta=['source_uid', 'media_type', 'source_path', 'status', 
              ['matches', 'uid'], 
              ['matches', 'artist'],
              ['matches', 'album'],
              ['matches', 'title'],
              ['matches', 'type'],
              ['matches', 'imdb_id'],
              ['matches', 'year'],
        ],
        errors='ignore'
    )
    df = df.rename(
        columns={
            'source_uid': 'que_uid', 
            'matches.uid': 'ref_uid',
            'matches.artist': 'artist',
            'matches.album': 'album',
            'matches.title': 'title',
            'matches.type': 'type',
            'matches.imdb_id': 'imdb_id',
            'matches.year': 'year',
        },
    )
    
    return df

In [21]:
new_df =  load_vertx_data_using_json_normalize(JSON_FILE)
new_df.head()

,duration,que_offset,ref_offset,que_uid,media_type,source_path,status,ref_uid,artist,album,title,type,imdb_id,year
0,24.9375,0.0,18.000,1758730481226206085,audio,sample.mp4,succeeded,6475547275973858650,Black Eyed Peas; Papa Roach,Elephunk,Anxiety,music,NaN,NaN
1,24.9375,0.0,3500.625,1758730481226206085,audio,sample.mp4,succeeded,1566530810344932800,NaN,NaN,You Got Served,movie,365957,2004
2,21.0000,0.0,8608.500,1758730481226206085,video,sample.mp4,succeeded,1342729426672482861,NaN,NaN,Troy,movie,332452,2004
